# This notebook will be used mainly for the capstone project

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Part One of Capstone : Explain your problem and your data

### 1. Introduction/Business Problem

Which city is the best city to live in?  This project will compare two cities, Toronto and New York City.  Both cities are very diverse and are the financial capitals of their respective countries. When the two are compared, which one will be considered the best city to live in?  For someone who is looking between New York City and Toronto for a job relocation, for example, this is an important question.  Some things to consider for finding a new home are safety, housing market, and amenities such as restaurants, parks, and entertainment.  And within both cities, which neighborhood is the best to live in?  This project aims to provide insight and answer these questions based on the parameters. 



### 2. Data

In order to answer these questions, we will need to pull data for each aspect of the search.  The data for the project will include geographical data such as neighbhorhoods and boroughs, crime data, housing data, and then data collected from Foursquare for restaurants, parks, and entertainments for both New York City and Toronto.  The geographical data will need to include neighborhoods, boroughs, and postal codes for both cities in order to use maps and segment into neighbhorhoods for the rest of the data for this project.

The crime data will need to include neighborhoods/boroughs, and lists of crime categories such as assault and theft for both cities.  This will provide insight for the relative safety of both cities both on the city-wide level and neighborhood level, which can be shown in a choropleth map.

Housing data is also important in terms of whether or not you can afford to live in the cities.  The housing data needs to include renting prices, costs of houses, and preferably cost of living and organizes by neighborhoods or boroughs.

The data on amenities such as restaurants, parks, and entertainment (movie theaters, for example) will be found and collected using Foursquare.  This is important especially on the neighbhorhood level to make a decision on which neighborhood is the best to live in based on the amenities available.

By using these data, the project aims to accumulate a score for each neighborhood and an overall score for both cities.  

#### Sources for the Capstone Project Data

GEOGRAPHICAL DATA:
The geographical data for Toronto: https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050 
New York City data will be  pulled from Geopy

CRIME:
Crime Data for New York - https://www1.nyc.gov/assets/nypd/downloads/excel/analysis_and_planning/historical-crime-data/seven-major-felony-offenses-2000-2019.xls.  This is the overal view by types of felonies and by year.
https://www.kaggle.com/adamschroeder/crimes-new-york-city/download  -> the 2014 crime data set for all New York Boroughs found from https://www.kaggle.com/adamschroeder/crimes-new-york-city#Crime_Column_Description.csv

Crime data for Toronto came from the site, http://data.torontopolice.on.ca/pages/open-data
 - Assault: https://opendata.arcgis.com/datasets/a5029dcd67214cc594a500865f868284_0.geojson
 -Auto Theft:https://opendata.arcgis.com/datasets/ca9730d5840343f7bfa3f5ffd16c2f0a_0.geojson
 -Break and Enter:https://opendata.arcgis.com/datasets/8ab59b498f6d4eae8ec631a287550376_0.geojson
 -Homicide:https://opendata.arcgis.com/datasets/7826a3ef2eff4d64a7f70e909de007b5_0.geojson
 -Robbery:https://opendata.arcgis.com/datasets/9115accc55f24938b4eb573dd222c33b_0.geojson
 -Theft Over: https://opendata.arcgis.com/datasets/19d66b66abe749e7b5be1a86e272f8ea_0.geojson


HOUSING PRICES:
Housing prices for New York
Housing dataset (rental): ('C:\Users\Amanda\Desktop\NYC Housing Data\clean2.csv') from https://www.kaggle.com/ted8080/nyc-housing-dataset-extrinsic-factors

Housing prices for Toronto
Apartmental rentals : ('C:\Users\Amanda\Desktop\Toronto Housing Data\Toronto Apartment Rental\Toronto_apartment_rentals_2018') from https://www.kaggle.com/rajacsp/toronto-apartment-price 
House sales : ('C:\Users\Amanda\Desktop\Toronto Housing Data\Ontario House Sales\properties.csv') from https://www.kaggle.com/mnabaee/ontarioproperties

AMENITIES:
Restaurants
Boutiques
Entertainmnet (Movie Theaters, Bowling, Bars, etc)
Will be found on Foursquare

# Part Two of Capstone: The Data Manipulation and Analysis

## Pull All The Data and Preprocess Them

In [3]:
#import all the libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab *already downloaded
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [7]:
#Toronto Geographical Data on Neighbhorhoods/Boroughs
Tor_url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'
Tor_df = pd.read_html(Tor_url)
Tor_df_postcodes=Tor_df[0]
print('imported dataframe has', Tor_df_postcodes['Postcode'].count(), "postcodes entriess")
Tor_df_postcodes.head(10)
#df_postcodes.dtypes

#New York Geographical Data on Neighborhoods/Boroughs
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

##Transform the New York Data into a dataframe
 # define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

 # instantiate the dataframe
NY_df = pd.DataFrame(columns=column_names)
 #put New York Data in dataframe
for data in NY_df:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NY_df = NY_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

imported dataframe has 287 postcodes entriess
Data downloaded!


TypeError: string indices must be integers

In [ ]:
#Preprocessing
#Toronto - Get rid of the "Not assigned" / Replace
#df_postcodes["Borough"].replace("nan", "Not assigned", inplace=True)
#df_postcodes["Neighbourhood"].replace("nan", "Not assigned", inplace=True)

Tor_df_postcodes = df_postcodes[df_postcodes["Borough"] != "Not assigned"]
#df_postcodes.dropna(subset=["Borough"], axis=0, inplace=True)

#Replace the N/A values of Neighborhood with the Borough Value
Tor_df_postcodes["Neighbourhood"].replace("Not assigned", df_postcodes["Borough"], inplace=True)

Tor_df_postcodes.head(20)

#Now combine all the neighbourhoods with the same postal codes
df = df_postcodes.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list)
df = df.sample(frac=1).reset_index()
df["Neighbourhood"] = df["Neighbourhood"].str.join(', ')
df.head(10)

## Use geopy library to get the latitude and longitude values of New York City and Toronto

In [4]:
#Toronto
#!pip install geocoder
#import geocoder

Torlat_list=[]
Torlng_list=[]
for i in range(df.shape[0]):
    address='{}, Toronto, Ontario'.format(Tor_df.at[i,'Postcode'])
    g=geocoder.arcgis(address)
    Torlat_list.append(g.latlng[0])
    Torlng_list.append(g.latlng[1])
    

#df.drop(['Postal Code_x', 'Latitude_x', 'Longitude_x','Postal Code_y'],axis=1, inplace=True)
Tor_df.drop(['Postal Code_y'], axis=1, inplace=True)
Tor_df.rename(columns= {'Latitude_y':'Latitude', 'Longitude_y':'Longitude'}, inplace=True)

Tor_df.head(10)

Toraddress = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
Torlocation = geolocator.geocode(Toraddress)
Torlatitude = location.latitude
Torlongitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#New York
NYaddress = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
NYlocation = geolocator.geocode(NYaddress)
NYlatitude = location.latitude
NYlongitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

## Maps

In [5]:
#Toronto
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[Torlatitude, Torlongitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(Tor_df['Latitude'], Tor_df['Longitude'], Tor_df['Borough'], Tor_df['Neighbourhood']):
    label ='{}, {}'.format(neighbourhood, borough) 
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
#New York
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[NYlatitude, NYlongitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NY_df['Latitude'], NY_df['Longitude'], NY_df['Borough'], NY_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork